In [ ]:
#r "nuget: Azure.AI.OpenAI, 2.1.0"
#r "nuget: DotNetEnv, 2.5.0"

using Azure;

using DotNetEnv;

using System.IO;
using System.Text.Json;
using System.ClientModel;

using Azure.AI.OpenAI;
using Azure.AI.OpenAI.Chat;

using OpenAI.Chat;

using OpenAI.Embeddings;

In [ ]:
string _configurationFile = @"../../../configuration/.env";
Env.Load(_configurationFile);
    
string oAiApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY") ?? "AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT") ?? "AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME") ?? "CHATCOMPLETION_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME") ?? "EMBEDDING_DEPLOYMENTNAME not found";
    
AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
AzureOpenAIClient openAIClient = new AzureOpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);
    
Console.WriteLine($"OpenAI Client created: {oAiEndpoint} with: {chatCompletionDeploymentName} and {embeddingDeploymentName} deployments");

In [ ]:
async Task<float[]> GetEmbeddingAsync(AzureOpenAIClient _openAIClient,string textToBeVecorized)
{
    // Prepare the embeddings options with the user story\n",
    EmbeddingClient embeddingClient = _openAIClient.GetEmbeddingClient(embeddingDeploymentName);
    ClientResult<OpenAIEmbedding> embeddingResult = await embeddingClient.GenerateEmbeddingAsync(textToBeVecorized);   
    float[] response = embeddingResult?.Value?.ToFloats().ToArray() ?? new float[0];
    return response;
}

In [ ]:
    float [] xx = await GetEmbeddingAsync(openAIClient,"sample text to embed");
    Console.WriteLine($"Embedding: {xx.Length}");

In [ ]:
async Task<string> CallOpenAI(AzureOpenAIClient _openAIClient, string prompt, string systemMessage, bool jsonResponse = true)
{
    // Get the chat client (using your deployment or model name)
    ChatClient chatClient = _openAIClient.GetChatClient(chatCompletionDeploymentName);

    ChatCompletionOptions chatComletionOptions = new ChatCompletionOptions(){
        MaxOutputTokenCount = 150,
        Temperature = 0.7f,
        TopP = 1.0f,
        FrequencyPenalty = 0.7f,
        PresencePenalty = 0.7f,

    };

    chatComletionOptions.ResponseFormat = jsonResponse ? ChatResponseFormat.CreateJsonObjectFormat() : ChatResponseFormat.CreateTextFormat();

    // Prepare your messages
    ChatMessage[] messages = new ChatMessage[]
    {
        new SystemChatMessage(systemMessage),
        new UserChatMessage(prompt)
    };

    // Call the chat completions endpoint with parameters directly
    ChatCompletion completions = await chatClient.CompleteChatAsync(        
    messages: messages, 
    options: chatComletionOptions);

    // Get the text from the first completion choice
    // var resp = completions.Content[0];
    
    string result = completions.Content[0].Text;
    return result;
}

In [ ]:
string systemMessage = 
@"You are an AI assistant that only answers wrong answers 
but stay in the domain of the question, 
and keep your answers short. 
For example, if the user asks for true or false, 
answer with a true or false statement. try to be funny or absurd. 
Always respond in the domain of the question.";

// Call your OpenAI method, presumably an async method that returns a string response
var resp = await CallOpenAI(openAIClient, 
    "who manufacures iPhone",
    systemMessage, false);

// Print the result
Console.WriteLine(resp);